In [1]:
import tensorflow as tf

sess = tf.Session()
print sess

In [2]:
#cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(300) for i in range(3)]) 
cell = tf.contrib.rnn.LSTMCell(300)
# cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=0.9)
print cell
init_state = cell.zero_state(100, tf.float32)
print init_state

batch_size = 100
seq_len = 20

x = tf.placeholder(tf.int32, [batch_size, seq_len])
#tf.get_variable_scope().reuse = True
embedding_matrix = tf.get_variable('embedding', [10000, 256]) # |V| = 10000, emb_dim= 256
inputs = tf.nn.embedding_lookup(embedding_matrix, x)
print inputs

LSTMStateTuple(c=<tf.Tensor 'LSTMCellZeroState/zeros:0' shape=(100, 300) dtype=float32>, h=<tf.Tensor 'LSTMCellZeroState/zeros_1:0' shape=(100, 300) dtype=float32>)
Tensor("embedding_lookup:0", shape=(100, 20, 256), dtype=float32)


In [4]:
?tf.layers.dense

In [3]:
outputs, state = tf.nn.dynamic_rnn(cell, inputs = inputs, initial_state=init_state)

print outputs
print '----'
print state

Tensor("rnn/transpose:0", shape=(100, 20, 300), dtype=float32)
----
LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(100, 300) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(100, 300) dtype=float32>)


In [21]:
logit = tf.layers.dense(state[1], 10000)
print logit

target = tf.placeholder(tf.int32, [batch_size, 10000])
print target

loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=target))
print loss

Tensor("dense_12/BiasAdd:0", shape=(100, 10000), dtype=float32)
Tensor("Placeholder_6:0", shape=(100, 10000), dtype=int32)
Tensor("Sum:0", shape=(), dtype=float32)


In [37]:
import numpy as np

xx = np.random.randint(0, 10000, size=(100, 20))
y = np.random.randint(0, 10000, size=100)
yy = np.zeros((100, 10000), dtype=np.float16)
yy[range(100), y] = 1
print yy[0:2].sum()

2.0


In [38]:
#sess.run(tf.global_variables_initializer())
sess.run(loss, feed_dict={x:xx, target:yy})

921.04865

In [39]:
decoder_cell = tf.nn.rnn_cell.LSTMCell(256)
print decoder_cell

In [41]:
attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(300, outputs) # 300 is encoder hidden size
decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism)
print decoder_cell

In [43]:
helper = tf.contrib.seq2seq.TrainingHelper(decoder_inputs_emb, decoder_lengths, time_major=True)
decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, outputs)
outputs, _ = tf.contrib.seq2seq.dynamic_decode(decoder,...)
logits = outputs.rnn_output

SyntaxError: invalid syntax (<ipython-input-43-047b6d603a71>, line 3)

In [44]:
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=decoder_outputs, logits=logits)
train_loss = tf.reduce_mean(loss * target_weights)

NameError: name 'decoder_outputs' is not defined